# Working with NBA live data...
Libraries supporting live data do not include support for Pandas. Note: Any call to `{endpoint}.{Class}()` will perform a request. Example: `scoreboard.ScoreBoard()`. In order to avoid multiple requests, set `{endpoint}.{Class}()` to a variable. See sample code below.

## Today's Score Board


In [1]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2021-01-18
0022000207: Spurs vs. Trail Blazers @ 2021-01-18 15:00:00-05:00
0022000205: Timberwolves vs. Hawks @ 2021-01-18 14:30:00-05:00
0022000203: Magic vs. Knicks @ 2021-01-18 12:00:00-05:00
0022000204: Cavaliers vs. Wizards @ 2021-01-18 14:00:00-05:00
0022000208: Suns vs. Grizzlies @ 2021-01-18 17:00:00-05:00
0022000209: Bucks vs. Nets @ 2021-01-18 19:30:00-05:00
0022000210: Mavericks vs. Raptors @ 2021-01-18 19:30:00-05:00
0022000206: Pistons vs. Heat @ 2021-01-18 20:00:00-05:00
0022000211: Rockets vs. Bulls @ 2021-01-18 20:00:00-05:00
0022000212: Warriors vs. Lakers @ 2021-01-18 22:00:00-05:00


## Box Score

In [6]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022000196') 

In [28]:
# Data Sets
box.game.get_dict()                    #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

{'gameId': '0022000196',
 'gameTimeLocal': '2021-01-17T13:00:00-05:00',
 'gameTimeUTC': '2021-01-17T18:00:00Z',
 'gameTimeHome': '2021-01-17T13:00:00-05:00',
 'gameTimeAway': '2021-01-17T13:00:00-05:00',
 'gameEt': '2021-01-17T13:00:00-05:00',
 'duration': 134,
 'gameCode': '20210117/NYKBOS',
 'gameStatusText': 'Final',
 'gameStatus': 3,
 'regulationPeriods': 4,
 'period': 4,
 'gameClock': 'PT00M00.00S',
 'attendance': 0,
 'sellout': '0',
 'arena': {'arenaId': 17,
  'arenaName': 'TD Garden',
  'arenaCity': 'Boston',
  'arenaState': 'MA',
  'arenaCountry': 'US',
  'arenaTimezone': 'America/New_York'},
 'officials': [{'personId': 2882,
   'name': 'Sean Wright',
   'nameI': 'S. Wright',
   'firstName': 'Sean',
   'familyName': 'Wright',
   'jerseyNum': '4',
   'assignment': 'OFFICIAL1'},
  {'personId': 201638,
   'name': 'Brent Barnaky',
   'nameI': 'B. Barnaky',
   'firstName': 'Brent',
   'familyName': 'Barnaky',
   'jerseyNum': '36',
   'assignment': 'OFFICIAL2'},
  {'personId': 162752

In [32]:
# Getting Box Scores. 
# Note: home_team & away_team have the identicial data structure.
players = box.away_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in players:
    print(f.format(player_id=player['personId'],name=player['name'],points=player['statistics']['points']))

203493: Reggie Bullock: 11 PTS
203944: Julius Randle: 20 PTS
1629011: Mitchell Robinson: 8 PTS
1629628: RJ Barrett: 19 PTS
203901: Elfrid Payton: 9 PTS
203457: Nerlens Noel: 5 PTS
203085: Austin Rivers: 0 PTS
1630193: Immanuel Quickley: 17 PTS
1630167: Obi Toppin: 12 PTS
1628995: Kevin Knox II: 3 PTS
1628372: Dennis Smith Jr.: 1 PTS
1629033: Theo Pinson: 0 PTS
1629649: Ignas Brazdeikis: 0 PTS
201959: Taj Gibson: 0 PTS
1629607: Jared Harper: 0 PTS
202692: Alec Burks: 0 PTS
1628373: Frank Ntilikina: 0 PTS


## Play By Play Data

In [50]:
# Query nba.live.endpoints for the score board of GameID 0022000180 = NYK vs BOS
# Simple PlayByPlay Loop demonstrating data usage
from nba_api.live.nba.endpoints import playbyplay
from nba_api.stats.static import players

pbp = playbyplay.PlayByPlay('0022000196')
line = "{action_number}: {period}:{clock} {player_id} ({action_type})"
actions = pbp.get_dict()['game']['actions'] #plays are referred to in the live data as `actions`
for action in actions:
    player_name = ''
    player = players.find_player_by_id(action['personId'])
    if player is not None:
        player_name = player['full_name']
    print(line.format(action_number=action['actionNumber'],period=action['period'],clock=action['clock'],action_type=action['actionType'],player_id=player_name))

2: 1:PT12M00.00S  (period)
4: 1:PT11M58.00S Grant Williams (jumpball)
7: 1:PT11M47.00S Julius Randle (foul)
9: 1:PT11M47.00S Grant Williams (freethrow)
10: 1:PT11M47.00S  (rebound)
11: 1:PT11M47.00S Grant Williams (freethrow)
12: 1:PT11M45.00S Julius Randle (rebound)
13: 1:PT11M25.00S RJ Barrett (turnover)
14: 1:PT11M25.00S Kemba Walker (steal)
15: 1:PT11M21.00S Kemba Walker (turnover)
16: 1:PT11M07.00S RJ Barrett (2pt)
17: 1:PT11M06.00S Mitchell Robinson (rebound)
18: 1:PT11M06.00S Mitchell Robinson (2pt)
19: 1:PT10M53.00S Tristan Thompson (2pt)
21: 1:PT10M26.00S RJ Barrett (3pt)
22: 1:PT10M24.00S Reggie Bullock (rebound)
23: 1:PT10M17.00S Julius Randle (3pt)
24: 1:PT10M01.00S Marcus Smart (3pt)
25: 1:PT09M58.00S Elfrid Payton (rebound)
26: 1:PT09M50.00S Elfrid Payton (2pt)
27: 1:PT09M50.00S  (timeout)
28: 1:PT09M35.00S Kemba Walker (3pt)
29: 1:PT09M29.00S RJ Barrett (rebound)
30: 1:PT09M20.00S Reggie Bullock (3pt)
31: 1:PT09M18.00S Kemba Walker (rebound)
32: 1:PT09M09.00S Grant Willi

242: 2:PT04M59.00S Austin Rivers (substitution)
243: 2:PT04M59.00S Kevin Knox II (substitution)
244: 2:PT04M59.00S Jaylen Brown (substitution)
245: 2:PT04M59.00S RJ Barrett (substitution)
246: 2:PT04M59.00S Reggie Bullock (substitution)
248: 2:PT04M57.00S Semi Ojeleye (jumpball)
251: 2:PT04M57.00S Mitchell Robinson (turnover)
252: 2:PT04M57.00S Tristan Thompson (steal)
253: 2:PT04M48.00S Marcus Smart (2pt)
254: 2:PT04M44.00S Javonte Green (rebound)
255: 2:PT04M38.00S Marcus Smart (2pt)
256: 2:PT04M36.00S Obi Toppin (rebound)
257: 2:PT04M26.00S Immanuel Quickley (2pt)
271: 2:PT04M09.00S Jaylen Brown (turnover)
272: 2:PT04M09.00S Reggie Bullock (steal)
262: 2:PT04M05.00S Marcus Smart (foul)
264: 2:PT04M05.00S RJ Barrett (freethrow)
265: 2:PT04M05.00S  (rebound)
266: 2:PT04M05.00S Semi Ojeleye (substitution)
267: 2:PT04M05.00S Obi Toppin (substitution)
268: 2:PT04M05.00S Grant Williams (substitution)
269: 2:PT04M05.00S Julius Randle (substitution)
270: 2:PT04M05.00S RJ Barrett (freethrow)

501: 4:PT12M00.00S Javonte Green (substitution)
502: 4:PT12M00.00S Jeff Teague (substitution)
503: 4:PT12M00.00S Tristan Thompson (substitution)
504: 4:PT12M00.00S Marcus Smart (substitution)
505: 4:PT12M00.00S Jaylen Brown (substitution)
506: 4:PT12M00.00S  (period)
507: 4:PT11M48.00S Tristan Thompson (2pt)
508: 4:PT11M48.00S Obi Toppin (block)
509: 4:PT11M47.00S  (rebound)
510: 4:PT11M47.00S  (stoppage)
511: 4:PT11M43.00S Tristan Thompson (foul)
513: 4:PT11M43.00S Tristan Thompson (turnover)
514: 4:PT11M32.00S Obi Toppin (2pt)
516: 4:PT11M19.00S Tristan Thompson (2pt)
517: 4:PT11M18.00S Jaylen Brown (rebound)
518: 4:PT11M16.00S Jaylen Brown (2pt)
560: 4:PT11M06.00S Immanuel Quickley (turnover)
561: 4:PT11M06.00S Grant Williams (steal)
562: 4:PT11M06.00S Grant Williams (turnover)
563: 4:PT11M06.00S Immanuel Quickley (steal)
519: 4:PT10M56.00S Immanuel Quickley (3pt)
520: 4:PT10M54.00S Marcus Smart (rebound)
521: 4:PT10M40.00S Marcus Smart (3pt)
522: 4:PT10M38.00S  (rebound)
523: 4:PT1